In [34]:
!pip install gephipy

In [35]:
#
# Create a Gephy workspace
#
from gephipy import gephipy

workspace = gephipy.create_workspace()


In [36]:
#
# Load a graph in Gephy
#
from gephipy import gephipy
import networkx as nx

# Create a random graph with NetworkX
graphX = nx.erdos_renyi_graph(500,0.01)

# Load it in Gephi
gephipy.networkx_to_gephi(workspace, graphX)

print("Import done!")

Import done!


In [37]:
#
# Compute some metrics
#
from gephipy import gephipy
from org.gephi.statistics.plugin import GraphDistance, Modularity

# Get the Gephi graph model from the current workspace
graphModel = gephipy.get_graph_model(workspace)

# Louvain
modularity = Modularity()
modularity.execute(graphModel)

# Betweeness centrality
centrality = GraphDistance()
centrality.setDirected(True)
centrality.execute(graphModel)

print("Metrics done!")

Metrics done!


In [38]:
#
# Apply appearance
#
from gephipy import gephipy

from org.openide.util import Lookup
from org.gephi.appearance.api import AppearanceController
from org.gephi.appearance.plugin import RankingNodeSizeTransformer, PartitionElementColorTransformer
from org.gephi.appearance.plugin.palette import PaletteManager

appearanceController = Lookup.getDefault().lookup(AppearanceController)
appearanceModel = appearanceController.getModel()

# Size Make node size based on centrality
centralityColumn = graphModel.getNodeTable().getColumn(GraphDistance.BETWEENNESS)
centralityRanking = appearanceModel.getNodeFunction(centralityColumn, RankingNodeSizeTransformer)
centralityTransformer = centralityRanking.getTransformer()
centralityTransformer.setMinSize(10)
centralityTransformer.setMaxSize(100)
appearanceController.transform(centralityRanking)

# Color by community
communityColumn = graphModel.getNodeTable().getColumn(Modularity.MODULARITY_CLASS)
colorPartition = appearanceModel.getNodeFunction(communityColumn, PartitionElementColorTransformer)
partition = colorPartition.getPartition()
palette = PaletteManager.getInstance().generatePalette(partition.size(graphModel.getGraph()))
partition.setColors(graphModel.getGraph(), palette.getColors())
appearanceController.transform(colorPartition)

print("Appearance done!")

Appearance done!


In [39]:
#
# Run Layouts
#
from gephipy import gephipy

from org.gephi.layout.plugin.forceAtlas2 import ForceAtlas2Builder
from org.gephi.layout.plugin.random import Random
from org.gephi.layout.plugin.noverlap import NoverlapLayoutBuilder

# Random layout
random = Random().buildLayout()
random.setGraphModel(gephipy.get_graph_model(workspace))
random.initAlgo()
random.goAlgo()
random.endAlgo()

# FA2 layout
fa2 = ForceAtlas2Builder().buildLayout()
fa2.setGraphModel(gephipy.get_graph_model(workspace))
fa2.resetPropertiesValues()
fa2.initAlgo()
for x in range(1000):
  fa2.goAlgo()
fa2.endAlgo()

# Noverlap layout
noverlap = NoverlapLayoutBuilder().buildLayout()
noverlap.setGraphModel(gephipy.get_graph_model(workspace))
noverlap.initAlgo()
noverlap.endAlgo()

print("Layouts done!")

Layout done!


In [46]:
#
# Export your graph / visualize your graph
#
from gephipy import gephipy
from IPython.display import display,HTML

# Transform it to graphX
graphx = gephipy.gephi_to_networkx(workspace)

# Export your graph
gephipy.export_gexf(workspace, "my-gephi-graph.gexf")
gephipy.export_pdf(workspace, "my-gephi-graph.pdf")

gephipy.export_svg(workspace, "my-gephi-graph.svg")
with open("./my-gephi-graph.svg", 'r') as file_svg:
  svg = file_svg.read()
  html_code = f"""
    <script src="//bumbu.me/svg-pan-zoom/dist/svg-pan-zoom.js"></script>
    <div id="svg-container">
    {svg}
    </div>
    <script>
    var svgElement = document.getElementById("svg-container").children[0];
    svgElement.style.width = "100%";
    svgElement.style.height = "500px";
    svgPanZoom(svgElement, {{
        zoomEnabled: true,
        controlIconsEnabled: true
    }});
    </script>
  """

  display(HTML(html_code))

print("Exports done!")

Exports done!
